<a href="https://colab.research.google.com/github/the-bucketless/glicko/blob/main/glicko_faceoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Glicko for Faceoffs

In [1]:
from functools import partial
from hyperopt import fmin, hp, STATUS_OK, tpe, Trials
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

## Glicko

This is an implementation of Glicko for rating players. It may be a one-time use in which case I've definitely abstracted things more than necessary, but that seems to be something I do.

Some of the different variants:  
http://www.glicko.net/glicko/glicko.pdf  
http://www.glicko.net/glicko/glicko2.pdf  
http://www.glicko.net/glicko/glicko-boost.pdf

This particular implementation follows the original formulas, though it can be updated for the others.  
  
It also includes some extra parameters that I haven't used, which may give you a few into some of the thoughts I had before settling on this version of things.

In [2]:
class Player:
    def __init__(self, id, rating, rating_deviation):
        self.id = id
        self.rating = rating
        self.rating_deviation = rating_deviation
        self.n_matches = 0
        self.total_score = 0
        self.history = [("Starting", rating, rating_deviation, 0, 0)]

    def update(self, rating, rating_deviation, match=None):
        self.rating = rating
        self.rating_deviation = rating_deviation

        if match is not None:
            self.n_matches += len(match.scores)
            self.total_score += np.sum(match.scores)
            
            self.history.append(
                (
                    match.rating_period_id, 
                    self.rating, 
                    self.rating_deviation,
                    self.n_matches,
                    self.total_score,
                )
            )


class Match:
    def __init__(self, player, rating_period_id=None):
        self.rating = player.rating
        self.rating_deviation = player.rating_deviation
        self.rating_period_id = rating_period_id

        self.opponent_ids = []
        self.opponent_ratings = []
        self.opponent_deviations = []
        self.adjustments = []
        self.scores = []

    def append(self, opponent, adjustments, score):
        self.opponent_ids.append(opponent.id)
        self.opponent_ratings.append(opponent.rating)
        self.opponent_deviations.append(opponent.rating_deviation)
        self.adjustments.append(adjustments)
        self.scores.append(score)

In [3]:
def make_iterable(arr):
    try:
        iter(arr)
        return arr
    except TypeError:
        return [] if arr is None else [arr]


class Glicko:
    """
    Glicko rating system for estimating the relative skill of players.

    Typical use will involve creating a Glicko object and running process on
    a dataset.
        glicko = Glicko()
        glicko.process(data)

    Attributes:
        _players: dict {str: Player}
            List of players accessed by their ID.

        starting_rating: numeric
            Rating to give players when added to the player list.

        starting_deviation: numeric
            Rating deviation to give players when added to the player list.

        c: numeric
            Value to use in update step for every player's rating deviation.

        etas: numpy array
            Value(s) of adjustment(s) to expected score calculations.  Will be
            multiplied by values passed to adjustments parameters.

        new_player_rating: numeric
            Rating to give players when added to the player list and 
            has_new_players is True in the associated method calls.

        new_player_deviation: numeric
            Rating deviation to give players when added to the player list and 
            has_new_players is True in the associated method calls.

        deviation_update_function: function
            Function to call when updating players' deviations.  Should take
            a Player and a numeric as parameters and return a numeric.

            Default is:
                RD = (RD**2 + c**2)**0.5

        _player_match_queue: dict {str: Match}
            A queue of matches for each player that need to be processed for
            the current rating period.  Accessed by the player's ID.
    """

    q = np.log(10) / 400

    def __init__(
        self, 
        starting_rating=1500, starting_deviation=350,
        c=15, etas=None,
        new_player_rating=1500, new_player_deviation=350,
        players=None, 
        deviation_update_function=None,
    ):
        """
        players: iterable of Player or None (default None)
            Previously created players to be included in the model.  
            Expects IDs in the players' attributes to be unique.
        """

        self._players = (
            {} 
            if players is None 
            else {player.id: player for player in make_iterable(players)}
        )
        
        self.starting_rating = starting_rating
        self.starting_deviation = starting_deviation

        self.c = c
        
        self.etas = np.array([] if etas is None else etas)

        self.new_player_rating = new_player_rating
        self.new_player_deviation = new_player_deviation

        self._player_match_queue = {}

        self.deviation_update_function = (
            (lambda player, c: (player.rating_deviation**2 + c**2)**0.5)
            if deviation_update_function is None
            else deviation_update_function
        )

    def _create_new_player(self, player_id, use_new_player=False):
        """
        Helper function to create new players with correct starting ratings and
        deviations.

        Arguments:
            player_id: str

            use_new_player: bool (default False)
                When True, will use new_player_rating and new_player_deviation
                when adding new players to track.
                
                When False, will use starting_rating and starting_deviation.

        Returns:
            Player
        """

        if use_new_player:
            return Player(
                player_id, self.new_player_rating, self.new_player_deviation
            )
        else:
            return Player(
                player_id, self.starting_rating, self.starting_deviation
            )


    @staticmethod
    def _create_dataframe(
        data, rating_period_ids, player_ids, opponent_ids, scores, adjustments,
    ):
        """
        Places all the data into a pandas DataFrame with specific column names:
            rating_period_ids
            player_ids
            opponent_ids
            scores
            adjustment{number}
        """
        
        def rename_or_set(data, i, column_name, column_data):
            """
            If column is already in the data, rename it for consistency.
            Otherwise, add it to the end of the dataframe.
            """
            if isinstance(column_data, str):
                return data.rename(columns={column_data: column_name})
            elif column_data is None:
                return data.rename(columns={data.columns[i]: column_name})
            else:
                data[column_name] = column_data
                return data

        if data is None:
            data = pd.DataFrame()
        
        n_columns = len(data.columns)

        for i, (column_name, column_data) in enumerate(zip(
            ("rating_period_ids", "player_ids", "opponent_ids", "scores"),
            (rating_period_ids, player_ids, opponent_ids, scores),
        )):
            data = rename_or_set(data, i, column_name, column_data)

        if isinstance(adjustments, str):
            adjustments = [adjustments]
        elif adjustments is None:
            adjustments = data.columns[4:n_columns]

        for i, adjustment in enumerate(adjustments):
            data = rename_or_set(data, i + 4, f"adjustment{i}", adjustment)

        return data

    def get_player(self, player_id, use_new_player=False):
        return self._players.get(
            player_id, self._create_new_player(player_id, use_new_player)
        )

    def get_players(self):
        return list(self._players.values())

    def compute_g(self, rating_deviation):
        return 1 / (1 + 3 * self.q**2 * rating_deviation**2 / np.pi**2)**0.5

    def compute_expected_score(
        self, g_RD, player_rating, opponent_ratings, adjustments=None,
    ):
        adjustments = [] if adjustments is None else adjustments
        adjustment = (
            np.sum(adjustments * self.etas, axis=-1) 
            if len(adjustments) 
            else 0
        )

        match_diff = player_rating + adjustment - opponent_ratings
        
        return 1 / (1 + 10**(-g_RD * match_diff / 400))

    def compute_expected_outcome(
        self, player_ids=None, opponent_ids=None, players=None, opponents=None, 
        adjustments=None, use_new_player=False, 
    ):
        """
        Calculate the expected outcome of a matchup/matchups.

        Unlike in the update step, the value passed to the g function is a 
        combination of the two players' rating deviations.  In the update step,
        it is only based on the deviations of the opponents.

        Arguments:
            player_ids: str or iterable of str or None (default None)
                If neither player_ids nor players are provided, a new player 
                will be created.
            
            opponent_ids: str or iterable of str or None (default None)
                If neither opponent_ids nor opponents are provided, a new player 
                will be created.

            players: Player or iterable of Player or None (default None)
                If neither player_ids nor players are provided, a new player 
                will be created.

            opponent: Player or iterable of Player or None (default None)
                If neither opponent_ids nor opponents are provided, a new player 
                will be created.

            adjustments: iterable of numeric or None (default None)
                Adjustments made when calculating expected scores.
                eg) In chess, white has the advantage of playing first and this
                advantage should be factored in to the expected score.

                Adjustment values should typically be values of -1, 0, or 1 
                and are multiplied by the etas attribute to determine the 
                magnitude of the adjustment. The number of adjustments has to 
                agree with the number of elements in the etas attribute.

                When None, no adjustments are included.

            use_new_player: bool (default False)
                Only used when player_ids or opponent_ids are provided but not
                found in the player list.

                When True, will use new_player_rating and new_player_deviation
                when creating a new player.
                
                When False, will use starting_rating and starting_deviation.

        Returns:
            float
        """

        def get_ratings_and_deviations(players, ids):
            if players is None:
                players = [
                    self.get_player(id, use_new_player) 
                    for id in make_iterable(ids)
                ]
            else:
                players = make_iterable(players)

            ratings = []
            deviations = []
            for player in players:
                ratings.append(player.rating)
                deviations.append(player.rating_deviation)

            return np.array(ratings), np.array(deviations)

        player_ratings, player_deviations = get_ratings_and_deviations(
            players, player_ids
        )
        opponent_ratings, opponent_deviations = get_ratings_and_deviations(
            opponents, opponent_ids
        )

        adjustments = np.array([] if adjustments is None else adjustments)

        RD = (player_deviations**2 + opponent_deviations**2)**0.5
        g_RD = self.compute_g(RD)

        return self.compute_expected_score(
            g_RD, player_ratings, opponent_ratings, adjustments
        )

    def update(self):
        """ 
        Update every player's rating and rating deviation based on the data in
        the player match queue.
        """

        for player_id in list(self._player_match_queue):
            match = self._player_match_queue.pop(player_id)

            opponent_deviations = np.array(match.opponent_deviations)
            g_RD = self.compute_g(opponent_deviations)
            expected_scores = self.compute_expected_score(
                g_RD, match.rating, match.opponent_ratings, match.adjustments, 
            )

            grd_scores = g_RD**2 * expected_scores * (1 - expected_scores)
            d_squared = 1 / (self.q**2 * np.sum(grd_scores))

            rating_deviation = (
                (1 / (1 / match.rating_deviation**2 + 1 / d_squared))**0.5
            )
            rating = (
                match.rating 
                + rating_deviation**2 
                * self.q 
                * np.sum(g_RD * (match.scores - expected_scores))
            )

            self._players[player_id].update(rating, rating_deviation, match)

    def update_deviation(
        self, c=None, player_ids=None, players=None, update_function=None,
    ):
        """
        Update the rating deviation for players.
        
        Arguments:
            c: numeric or None (default None)
                If None, c attribute will be used.

            player_ids: iterable of str or None (default None)
                If both player_ids and players are None, will update every 
                player in the player list.

            players: iterable of Player or None (default None)
                If both player_ids and players are None, will update every 
                player in the player list.

            update_function: function or None (default None)
                A function that takes a Player and a numeric value as
                arguments and returns a float.

                Allows for customization of how players' deviations are
                updated.

                If not provided, the update_function attribute will be used.

        Returns:
            None
        """

        if update_function is None:
            update_function = self.deviation_update_function

        c = c or self.c

        if player_ids is None:
            players = (
                self._players.values() 
                if players is None
                else make_iterable(players)
            )
        else:
            player_ids = make_iterable(player_ids)
            players = [
                self._players[player_id] 
                for player_id in player_ids
                if player_id in self._players
            ]

        for player in players:
            rating_deviation = min(
                update_function(player, c), 
                self.starting_deviation,
                self.new_player_deviation,
            )
            player.update(player.rating, rating_deviation)

    def _process_matchup(
        self, player_id, opponent_id, score, adjustments=None,
        use_new_player=False, rating_period_id=None, 
    ):
        """
        Process a single matchup from one player's perspective.  This involves
        adding the player to the player list if not already present and
        adding the matchup to the player match queue.

        The opponent will not be added to the player list even when absent.

        Arguments:
            player_id: str

            opponent_id: str

            score: numeric

            adjustments: iterable or None (default None)
                Adjustments to be factored in to the expected score calculation.

            use_new_player: bool (default False)
                When True, will use new_player_rating and new_player_deviation
                when adding new players to track.
                
                When False, will use starting_rating and starting_deviation.

            rating_period_id: str or None (default None)
                ID to store in players' histories for current rating period.

        Returns:
            None
        """

        adjustments = np.array(make_iterable(adjustments))

        if player_id not in self._players:
            self._players[player_id] = self._create_new_player(
                player_id, use_new_player
            )

        opponent = self._players.get(
            opponent_id, self._create_new_player(opponent_id, use_new_player)
        )

        if player_id not in self._player_match_queue:
            self._player_match_queue[player_id] = Match(
                self._players[player_id], rating_period_id
            )
        self._player_match_queue[player_id].append(opponent, adjustments, score)

    def process(
        self, 
        data=None, rating_period_ids=None,
        player_ids=None, opponent_ids=None, scores=None, adjustments=None, 
        has_new_players=False, mirror_matches=True, has_deviation_update=True,
        verbose=0,
    ):
        """
        Update the ratings and rating deviations for players based on the 
        results of matches over rating periods.

        Arguments:
            data: pandas DataFrame or None (default None)
                When None, expects all required values to be passed to other
                parameters.

                When a pandas DataFrame, expects other parameters to be the
                names of the columns of the DataFrame containing their values
                or for the columns to be arranged:
                    period ID | player ID | opponent ID | scores
                With one column for each advantage to be included at the end.

            rating_period_ids: str or iterable or None (default None)
                IDs to distinguish different rating periods within the data.

                When data is None, should be an iterable of str.

                Otherwise, should be a str of the name of the column
                containing the rating periods.

                When None, should be the first column of data.

            player_ids: str or iterable of str or None (default None)
                When data is None, should be an iterable of str.

                Otherwise, should be a str of the name of the column
                containing player IDs.

                When None, should be the second column of data.

            opponent_ids: str or iterable of str or None (default None)
                When data is None, should be an iterable of str.

                Otherwise, should be a str of the name of the column
                containing opponent IDs.

                When None, should be the third column of data.

            scores: str or iterable of numeric or None (default None)
                When data is None, should be an iterable of numeric.

                Otherwise, should be a str of the name of the column
                containing the score for each matchup.

                When None, should be the fourth column of data.

            adjustments: iterable or None (default None)
                Adjustments made when calculating expected scores.
                eg) In chess, white has the advantage of playing first and this
                advantage should be factored in to the expected score.

                Adjustment values should typically be values of -1, 0, or 1 
                and are multiplied by the etas attribute to determine the 
                magnitude of the adjustment. The number of adjustments has to 
                agree with the number of elements in the etas attribute.

                When data is None, should contain in which each element is 
                an iterable of numeric values representing the weight of the
                adjustment.
                eg) [[0, 1, -1], [1, 0, -1]] would reflect that there are three
                games, each with two adjustments.

                When data is not None, should contain an interable of str in
                which each str is the name of the column of the DataFrame
                containing an adjustment to be factored in.

                When None, any columns in data beyond the fourth will be 
                considered adjustments.

            has_new_players: bool (default False)
                When True, will use new_player_rating and new_player_deviation
                when adding new players to track.
                
                When False, will use starting_rating and starting_deviation.

            mirror_matches: bool (default True)
                Whether or not to process both sides of the match.
                ie) If Player A plays Player B, match should be set up as both
                Player A vs Player B and Player B vs Player A.

                When True, the Player B vs Player A game will have adjustments 
                multiplied by -1 and scores will be 1 - scores.

                If False, match should be duplicated beforehand.

            has_deviation_update: bool (default True)
                Whether or not to update every player's deviation before
                processing the matchups.

            verbose: int (default 0)
                0: No progress bar.
                1: Progress bar for rating periods.
                2: Progress bar for matchups in each rating period.

        Returns:
            None
        """

        data = self._create_dataframe(
            data, rating_period_ids, player_ids, 
            opponent_ids, scores, adjustments, 
        )

        adjustment_columns = [c for c in data.columns if "adjustment" in c]

        rating_period_data = data.groupby("rating_period_ids")
        for period_id, period_data in tqdm(
            rating_period_data, 
            desc="Periods",
            disable=not verbose,
        ):
            if has_deviation_update:
                self.update_deviation()

            # Tried a different version of this that, instead of iterating over
            # the rows, iterated over a groupby of player_ids. That allowed
            # for vectorization of the update procedures.
            # It was a *lot* slower.
            for row in tqdm(
                period_data.itertuples(),
                total=len(data),
                desc="Matchups",
                disable=verbose < 2
            ):
                matchup_adjustments = np.array(
                    [getattr(row, c) for c in adjustment_columns]
                )
                
                self._process_matchup(
                    row.player_ids,
                    row.opponent_ids,
                    row.scores,
                    matchup_adjustments,
                    has_new_players,
                    row.rating_period_ids,
                )

                if mirror_matches:
                    self._process_matchup(
                        row.opponent_ids,
                        row.player_ids,
                        1 - row.scores,
                        -matchup_adjustments,
                        has_new_players,
                        row.rating_period_ids,
                    )

            self.update()

### A few simple tests

In [4]:
def get_players():
    return [
        Player(0, 1500, 200),
        Player(1, 1400, 30),
        Player(2, 1550, 100),
        Player(3, 1700, 300),
    ]


# Expected outcome test.

glicko = Glicko(players=get_players())
match_outcome = glicko.compute_expected_outcome(
    players=Player(4, 1400, 80),
    opponents=Player(5, 1500, 150)
)
assert(np.round(match_outcome, 3) == 0.376)


# Correct rating and deviation after processing one period.
# Also ensure Player A vs Player B gives same result as Player B vs Player A.

glicko_ab = Glicko(players=get_players())
glicko_ab.process(
    rating_period_ids=[0, 0, 0],
    player_ids=[0, 0, 0],
    opponent_ids=[1, 2, 3],
    scores=[1, 0, 0],
    has_deviation_update=False,
)

glicko_ba = Glicko(players=get_players())
glicko_ba.process(
    rating_period_ids=[0, 0, 0],
    player_ids=[1, 2, 3],
    opponent_ids=[0, 0, 0],
    scores=[0, 1, 1],
    has_deviation_update=False,
)

player_ab = glicko_ab.get_player(0)
player_ba = glicko_ba.get_player(0)
assert(np.round(player_ab.rating) == 1464)
assert(np.round(player_ab.rating_deviation, 1) == 151.4)
assert(player_ab.rating == player_ba.rating)
assert(player_ab.rating_deviation == player_ba.rating_deviation)

## Process NHL Data

Getting the [fastRhockey data](https://github.com/sportsdataverse/fastRhockey-data/tree/main/nhl) into the shape we need. The data returned has columns for:
- rating_period: The year and month the games took place in. Months with fewer than 26 teams active or 50 total games will get grouped in with the previous month or, in the case of the first month, the following month.
- home_player_id
- away_player_id
- home_win
- home_stick_down
    - 1: Home player puts stick down second.
    - -1: Away player puts stick down second.
- home_strong_side
    - 1: Home player on strong side vs away player on weak side.
    - 0: Both players on strong side or both players on weak side.
    - -1: Away player on strong side vs home player on weak side.
  
We need to work with player IDs rather than names because some players have different names in different games.

I may have been working on some pandas chaining and seeing if I could everything into one long chain (I could).

In [5]:
def process_nhl(year):
    column_names = [
        "rating_period", "home_player_id", "away_player_id",
        "home_win", "home_stick_down", "home_strong_side", 
    ]
    venues = ("home", "away")

    pbp_url = (
        "https://github.com/sportsdataverse/fastRhockey-data/blob/main/nhl/"
        f"pbp/parquet/play_by_play_{year}.parquet?raw=true"
    )

    schedule_url = (
        "https://github.com/sportsdataverse/fastRhockey-data/blob/main/nhl/"
        f"schedules/parquet/nhl_schedule_{year}.parquet?raw=true"
    )

    player_box_url = (
        "https://github.com/sportsdataverse/fastRhockey-data/blob/main/nhl/"
        f"player_box/parquet/player_box_{year}.parquet?raw=true"
    )

    pbp = pd.read_parquet(pbp_url)

    # Find direction each team is shooting (-1 to the left, 1 to the right).
    directions = (
        pbp
        .query("result_event_type_id in ('SHOT', 'MISS', 'BLOCK', 'GOAL')")
        .groupby(["game_id", "about_period", "team_id"])
        ["coordinates_x"]
        .agg(direction=lambda x: np.sign(np.median(x)))
        .replace(0, pd.NA)
    )

    def merge_directions(df):
        for venue in venues:
            df = (
                df
                .merge(
                    directions
                    .rename(columns={"direction": f"{venue}_direction"}),
                    left_on=["game_id", "about_period", f"{venue}_team_id"],
                    right_index=True,
                    how="left",
                )
            )

        return (
            df
            # In case a team didn't take a shot in a period.
            .fillna({
                "home_direction": -df.away_direction,
                "away_direction": -df.home_direction,
            })
        )


    player_box = (
        pd.read_parquet(player_box_url)
        [["player_id", "shoots_catches"]]
        .drop_duplicates()
    )

    def merge_handedness(df):
        for venue in venues:
            id_column = f"{venue}_player_id"
            df = (
                df
                .merge(
                    player_box
                    .rename(columns={
                        "player_id": id_column,
                        "shoots_catches": f"{venue}_player_shoots",
                    }),
                    on=id_column,
                    how="left",
                )
            )

        return df

    
    def venue_strong_side(df, venue):
        """
        Whether or not the center is on their strong side for both teams.
            Left-handed strong side:
                Direction to the right and y-coordinate >= 0.
                Direction to the left and y-coordinate < 0.
            Right-handed is the opposite.
        """

        return df.assign(**{
            f"{venue}_is_strong_side": np.where(
                df[f"{venue}_direction"] * (df.coordinates_y + 1) >= 0,
                df[f"{venue}_player_shoots"] == "L",
                df[f"{venue}_player_shoots"] == "R",
            )
            .astype(int)
        })

    return (
        pbp
        # Remove anything that isn't a faceoff.
        .query("result_event_type_id == 'FACEOFF'")
        # Get home and away teams from schedule data.
        .merge(
            pd.read_parquet(schedule_url)
            [["game_id", "home_team_id", "away_team_id"]],
            on="game_id",
            how="left",
        )
        # Sort by game date.
        .assign(
            about_date_time=lambda df_: df_.about_date_time.str.split("T").str[0]
        )
        .sort_values("about_date_time", kind="stable")
        .assign(
            # Get the month from the date.
            month=lambda df_: df_.about_date_time.str.split("-").str[1],
            n_teams=lambda df_: (
                df_.groupby("month")["team_id"].transform("nunique")
            ),
        )
        .assign(
            # Combine months in which fewer than 26 teams played.
            month=lambda df_: (
                df_.month.where(df_.n_teams > 26, pd.NA)
                .fillna(method="ffill")
                .fillna(method="bfill")
            ),
            n_games=lambda df_: (
                df_.groupby("month")["game_id"].transform("nunique")
            ),
        )
        .assign(
            # Combine months in which there were fewer than 50 games.
            month=lambda df_: (
                df_.month.where(df_.n_games > 49, pd.NA)
                .fillna(method="ffill")
                .fillna(method="bfill")
            ),
            # Combine year and month for use as rating period ID.
            rating_period=lambda df_: f"{year}-" + df_.month,
        )
        # Add directions for both teams.
        .pipe(merge_directions)
        .assign(
            # Track whether or not the home team won the faceoff.
            home_win=lambda df_: df_.team_id == df_.home_team_id,
            # Assign players to teams.
            home_player_id=lambda df_: np.where(
                df_.home_win,
                df_.player_id_1,
                df_.player_id_2
            ),
            away_player_id=lambda df_: np.where(
                df_.home_win,
                df_.player_id_2,
                df_.player_id_1
            )
        )
        # Add handedness of players from player_box data.
        .pipe(merge_handedness)
        .assign(
            # Player who puts their stick down second. Before the 2015/16 
            # season, the home team always had this advantage. Afterward, the 
            # player in the offensive zone has the advantage.
            home_stick_down=lambda df_: (
                np.where(df_.home_direction * df_.coordinates_x >= 0, 1, -1)
                if year > 2015
                else 1
            ),
        )
        # Find whether or not each player is on their strong side.
        # Haven't been able to get this to work as a dict comprehension in
        # an assign.
        .pipe(lambda df_: venue_strong_side(df_, "home"))
        .pipe(lambda df_: venue_strong_side(df_, "away"))
        # Strong side advantage:
        #    1: Strong vs weak
        #   -1: Weak vs strong
        #    0: Otherwise
        .assign(
            home_strong_side=lambda df_: np.sign(
                df_.home_is_strong_side - df_.away_is_strong_side
            ),
        )
        # Anything that's missing can be a 0.
        .fillna(0)
        # Cut down to only the columns that will be needed.
        [column_names]
        # Convert everything but the date to an int.
        .astype({c: int for c in column_names[1:]})
    )

## Process PHF Data

Getting the [fastRhockey data](https://github.com/sportsdataverse/fastRhockey-data/tree/main/phf) into the shape we need. The data returned has columns for:
- rating_period: The year and month the games took place in.
- home_player_id
- away_player_id
- home_win
- home_strength
    - 1: Home team has more skaters on than away team.
    - 0: Teams are at even-strength.
    - -1: Away team has more skaters on than home team.
  
I've kept the names of the columns consistent with the NHL data, though the IDs are actually the player names in this case.

In [6]:
def process_phf(year):
    pbp_url = (
        "https://github.com/sportsdataverse/fastRhockey-data/blob/main/phf/"
        f"pbp/parquet/play_by_play_{year}.parquet?raw=true"
    )

    schedule_url = (
        "https://github.com/sportsdataverse/fastRhockey-data/blob/main/phf/"
        f"schedules/parquet/phf_schedule_{year}.parquet?raw=true"
    )

    return (
        pd.read_parquet(pbp_url)
        # Get rid of everything but faceoffs.
        .query("play_type == 'Faceoff'")
        # Retrieve game dates from the schedule file.
        .merge(
            pd.read_parquet(schedule_url)
            [["game_id", "date_group"]],
            on="game_id",
            how="left",
        )
        # Sort by game data.
        .sort_values("date_group", kind="stable")
        .assign(
            # Use the year and the month as the rating period.
            rating_period=lambda df_: df_.date_group.astype(str).str[:7],
            # rating_period
            home_win=lambda df_: df_.team == df_.home_team,
            # Using names as IDs. Hopefully they're consistent from year to year.
            home_player_id=lambda df_: np.where(
                df_.home_win, df_.player_name_1, df_.player_name_2
            ),
            away_player_id=lambda df_: np.where(
                df_.home_win, df_.player_name_2, df_.player_name_1
            ),
            # Track numerical advantages for the home team.
            home_strength=lambda df_: np.sign(
                df_.home_skaters - df_.away_skaters
            ),
        )
        [[
            "rating_period", "home_player_id", "away_player_id",
            "home_win", "home_strength", 
        ]]
    )

## Hyperparameter Tuning

Anything that can apply to either league is up top here.  I've created a couple helper functions that can also be used in the results section.

There's run_glicko function includes an extra rating deviation for the end of each season.

Parameters that get tuned for both leagues:
- Starting deviation for players: I chose to restrict this to whole numbers.
- c$_{monthly}$: the c parameter for monthly deviation updates.
- c$_{annual}$: the c parameter for year-to-year updates.

In [7]:
universal_space = {
    "starting_deviation": hp.quniform("starting_deviation", 50, 500, 1),
    "c_monthly": hp.uniform("c_monthly", 0, 50),
    "c_annual": hp.uniform("c_annual", 0, 50),
}

In [8]:
# Taken from sklearn.
def binomial_deviance(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return -2 * np.mean(y_true * y_pred - np.logaddexp(0, y_pred))

In [9]:
def create_glicko(params):
    """ Helper function to create a Glicko object from hyperparameters. """

    eta_names = [param[:-4] for param in params.keys() if "eta" in param]

    glicko = Glicko(
        starting_deviation=params["starting_deviation"],
        c=params["c_monthly"],
        etas=[params[f"{eta_name}_eta"] for eta_name in eta_names],
    )

    return glicko


def run_glicko(
    glicko, params, data=None, years=None, 
    process_function=process_nhl, verbose=False,
):
    """ Helper function to call both when tuning and finding results. """

    if isinstance(data, pd.DataFrame):
        data = [data]

    if data is None:
        data = make_iterable(years)

    for year in tqdm(data, total=len(data), disable=not verbose):
        if not isinstance(year, pd.DataFrame):
            year = process_function(year)

        glicko.process(year)
        glicko.update_deviation(params["c_annual"])


def compute_loss(glicko, data, params, loss_function=binomial_deviance):
    """ Helper function to calculate loss. """

    eta_names = [param[:-4] for param in params.keys() if "eta" in param]
    eta_columns = [f"home_{eta_name}" for eta_name in eta_names]

    expected_outcomes = glicko.compute_expected_outcome(
        data["home_player_id"], data["away_player_id"],
        adjustments=data[eta_columns],
    )
    
    return loss_function(data["home_win"], expected_outcomes)

In [10]:
def objective(params, train_data, test_data, loss_function=binomial_deviance):
    glicko = create_glicko(params)
    run_glicko(glicko, params, train_data)

    loss = compute_loss(glicko, test_data, params, loss_function)

    return {
        "loss": loss,
        "params": params,
        "status": STATUS_OK,
    }

eval_loop by default saves a csv of the trial runs.

In [11]:
def eval_loop(
    league, max_evals, train_data, test_data, 
    space=None, write_trials_df=True,
):
    if max_evals == 0:
        # Based on 100 evals:
        #   Best NHL loss: 1.418628761081627
        #   Best PHF loss: 1.4170284821440358
        if league == "nhl":
            return (
                {
                    "c_annual": 43.420559997700344,
                    "c_monthly": 27.546896063782995,
                    "starting_deviation": 240,
                    "stick_down_eta": 0.5924466773314494,
                    "strong_side_eta": 183.31547435005967,
                },
                None
            )
        else:
            return (
                {
                    "c_annual": 4.843986454600442,
                    "c_monthly": 10.653388858842238,
                    "starting_deviation": 133,
                    "strength_eta": 199.7461164606941
                },
                None
            )

    trials = Trials()

    trials_objective = partial(
        objective, 
        train_data=train_data, 
        test_data=test_data,
    )

    best_params = fmin(
        fn=trials_objective,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=np.random.RandomState(42),
    )

    if write_trials_df:
        trials_df = pd.DataFrame(trials.trials)
        results = pd.json_normalize(trials_df.result)
        trials_df = trials_df.join(results)
        trials_df.to_csv(f"glicko_{league}_faceoffs_trials.csv", index=False)

    return best_params, trials

### NHL

When max_nhl_evals is set to 0, the model will default to the previously found best parameters on 100 trial runs. Tuning for that took about 10 minutes.

Parameters unique to the NHL set:
- $\eta_{stick\ down}$: the magnitude of the advantage of a player getting to put their stick down second.
- $\eta_{strong\ side}$: the magnitude of the advantage of a player on their strong side going against a player on their weak side.

The training set for the NHL consists of the 2014-2017 seasons with 2018 withheld as a validation set.

In [12]:
nhl_space = {
    **universal_space,
    "stick_down_eta": hp.uniform("stick_down_eta", 0, 200),
    "strong_side_eta": hp.uniform("strong_side_eta", 0, 200),
}

In [13]:
max_nhl_evals = 0    #@param {type: "number"}

Predicting 50/50 for every faceoff for comparison.

In [14]:
if max_nhl_evals > 0:
    nhl_train_data = [process_nhl(year) for year in range(2014, 2018)]
    nhl_test_data = process_nhl(2018)
    
    # Naive model.
    binomial_deviance(nhl_test_data["home_win"], 0.5)
else:
    nhl_train_data, nhl_test_data = None, None

In [15]:
nhl_best_params, nhl_trials = eval_loop(
    "nhl", max_nhl_evals, nhl_train_data, nhl_test_data, nhl_space, 
)

In [16]:
nhl_best_params

{'c_annual': 43.420559997700344,
 'c_monthly': 27.546896063782995,
 'starting_deviation': 240,
 'stick_down_eta': 0.5924466773314494,
 'strong_side_eta': 183.31547435005967}

### PHF

When max_nhl_evals is set to 0, the model will default to the previously found best parameters on 100 trial runs. Tuning for that took about 15 seconds.

Parameter unique to the PHF set:
- $\eta_{strength}$: the magnitude of the advantage of a team having a numerical advantage in skaters.

The training set for the NHL consists of the 2020 and 2021 seasons with 2022 withheld as a validation set.

In [17]:
# Choosing to use whole numbers as starting points for ratings and deviations.
phf_space = {
    **universal_space,
    "strength_eta": hp.uniform("strength_eta", 0, 200),
}

In [18]:
max_phf_evals = 0    #@param {type: "number"}

Predicting 50/50 for every faceoff for comparison.

In [19]:
if max_phf_evals > 0:
    phf_train_data = [process_phf(year) for year in (2020, 2021)]
    phf_test_data = process_phf(2022)

    # Naive model.
    binomial_deviance(phf_test_data["home_win"], 0.5)
else:
    phf_train_data = None
    phf_test_data = None

In [20]:
phf_best_params, phf_trials = eval_loop(
    "phf", max_phf_evals, phf_train_data, phf_test_data, phf_space, 
)

In [21]:
phf_best_params

{'c_annual': 4.843986454600442,
 'c_monthly': 10.653388858842238,
 'starting_deviation': 133,
 'strength_eta': 199.7461164606941}

## Results

Applying the previously found best parameters to every season to get the player rankings. Both leagues have csv files of the results saved.

### NHL

Because we're using the player IDs for the NHL, we need to grab some names from the player_box files to match them to.

In [22]:
player_names = (
    pd.concat([
        pd.read_parquet(
            "https://github.com/sportsdataverse/fastRhockey-data/blob/main/nhl/"
            f"player_box/parquet/player_box_{year}.parquet?raw=true"
        )
        .rename(columns={"player_full_name": "player_name"})
        [["player_id", "player_name"]]
        for year in tqdm(range(2011, 2023))
    ])
    .drop_duplicates("player_id")
)

  0%|          | 0/12 [00:00<?, ?it/s]

In [23]:
nhl_glicko = create_glicko(nhl_best_params)
run_glicko(
    nhl_glicko, nhl_best_params, years=range(2011, 2023), verbose=True,
)

  0%|          | 0/12 [00:00<?, ?it/s]

In [24]:
full_nhl_history = pd.concat(
    [
        pd.DataFrame(
            player.history, 
            columns=["date", "rating", "deviation", "n_faceoffs", "wins"]
        )
        .assign(player_id=player.id)
        .merge(player_names, on="player_id", how="left")
        for player in tqdm(nhl_glicko.get_players())
    ]
)
full_nhl_history.to_csv("nhl_faceoffs.csv", index=False)

  0%|          | 0/1632 [00:00<?, ?it/s]

In [25]:
full_nhl_history

,date,rating,deviation,n_faceoffs,wins,player_id,player_name
0,Starting,1500.000000,240.000000,0,0,8464977,Dainius Zubrus
1,2011-01,1555.310911,56.129699,64,38,8464977,Dainius Zubrus
2,2011-02,1575.458032,36.586663,144,88,8464977,Dainius Zubrus
3,2011-03,1552.356689,35.831367,190,114,8464977,Dainius Zubrus
4,2011-04,1549.652139,43.948228,194,116,8464977,Dainius Zubrus
...,...,...,...,...,...,...,...
1,2022-12,1327.976161,205.714780,1,0,8482062,Cole Smith
0,Starting,1500.000000,240.000000,0,0,8479362,Riley Tufte
1,2022-12,1393.041188,187.327025,2,1,8479362,Riley Tufte
0,Starting,1500.000000,240.000000,0,0,8481093,Rafael Harvey-Pinard


### PHF

In [26]:
phf_glicko = create_glicko(phf_best_params)
run_glicko(
    phf_glicko, phf_best_params, None, range(2020, 2023), process_phf, True,
)

  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
full_phf_history = pd.concat(
    [
        pd.DataFrame(
            player.history, 
            columns=["date", "rating", "deviation", "n_faceoffs", "wins"]
        )
        .assign(player_name=player.id)
        for player in tqdm(phf_glicko.get_players())
    ]
)
full_phf_history.to_csv("phf_faceoffs.csv", index=False)

  0%|          | 0/154 [00:00<?, ?it/s]

In [28]:
full_phf_history

,date,rating,deviation,n_faceoffs,wins,player_name
0,Starting,1500.000000,133.000000,0,0,Hanna Beattie
1,2019-10,1532.693561,52.605163,45,24,Hanna Beattie
2,2019-12,1539.352640,53.705578,47,25,Hanna Beattie
3,2020-02,1538.687334,53.228169,51,27,Hanna Beattie
4,2020-03,1523.349307,53.026633,53,27,Hanna Beattie
...,...,...,...,...,...,...
1,2022-03,1508.749887,117.217332,2,1,Amanda Conway
0,Starting,1500.000000,133.000000,0,0,Dana Trivigno
1,2022-03,1489.290496,55.908902,34,14,Dana Trivigno
0,Starting,1500.000000,133.000000,0,0,Teresa Vanisova
